# Assignment 1
You should submit the **UniversityNumber.ipynb** file and your final prediction file **UniversityNumber.test.out** to moodle. Make sure your code does not use your local files and that the results are reproducible. Before submitting, please **run your notebook and keep all running logs** so that we can check.

## 1 $n$-gram Language Model

In [7]:
!wget -O train.txt https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/lm/train.txt
!wget -O dev.txt https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/lm/dev.txt
!wget -O test.txt https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/lm/test.txt

'wget' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
'wget' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


'wget' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


### 1.1 Building vocabulary

#### Code

In [29]:
dataset = 'train'

# for unigram
from collections import defaultdict
unk = set()
uniCnt = defaultdict(int)
word_cnt = 0

# Count the number of occurrences of each word
with open(f'data/lm/{dataset}.txt', encoding='utf-8') as fd:
    lines = fd.readlines()
    for line in lines:
        words = line.split(' ')
        word_cnt += len(words)
        for word in words:
            uniCnt[word] += 1
    uniCnt['<s>'] = len(lines)
    uniCnt['</s>'] = len(lines)
    word_cnt += len(lines) * 2

print(len(uniCnt))
assert sum(uniCnt.values()) == word_cnt


53412


In [30]:
# calc unk and filter for oov

unk = dict(filter(lambda item: item[1] < 3, uniCnt.items()))
voc = dict(filter(lambda item: item[1] >= 3, uniCnt.items()))
voc['unk'] = sum(unk.values())
assert sum(voc.values()) == word_cnt

print(unk)

{'Haldane': 2, 'contact-intensive': 1, 'streamline': 2, 'carbon-adjustment': 1, 'asimilar': 1, 'Czeslaw': 2, 'Milosz': 2, 'ispoetry': 1, 'acquit': 1, 'Sides': 1, 'escalates': 1, 'Bolstered': 1, 'drained': 1, 'noyau': 1, 'dur': 1, 'scans': 1, 'cerebral': 2, 'cortex': 2, 'fructose': 2, 'limbic': 1, 'lizard': 1, '1.00': 2, 'indicts': 1, 'Troublesome': 1, 'Neighbors\n': 1, 'Down\n': 1, 'deadlocks': 2, 'inter-state': 2, 'government-enforced': 1, 'principal-agent': 2, 'nasal': 1, 'swabs': 1, '24-48': 1, 'relieves': 2, 'offtake': 1, 'disgruntled': 1, 'provincials': 1, 'Picot': 2, 'Iran-Hezbollah-Hamas': 1, 'strut': 2, 'Geoffrey': 2, 'Berman': 1, 'Manhattan-based': 1, 'Twenty-first-century': 1, 'Berezovsky': 2, 'Tree': 1, 'pro-saving': 1, 'IRAs': 1, '401Ks': 1, 'amongst': 2, 'MONI': 1, 'FREETOWN': 2, 'SIERRA': 2, 'LEONE': 2, 'Juergen': 1, 'mint': 2, 'idiosyncrasies': 2, 'Mandarin': 1, '445': 1, 'coliform': 1, '135-plus': 1, 'near-meltdown': 1, 'Elites': 2, 'conscientiously': 2, 'creatively': 1

In [38]:
# for bigram
from collections import defaultdict
biCnt = defaultdict(lambda: defaultdict(int))

# To calculate the occurrence count of wi-1 wi
with open(f'data/lm/{dataset}.txt', encoding='utf-8') as fd:
    lines = fd.readlines()
    for line in lines:
        words = line.split(' ')
        for i in range(1, len(words)):
            biCnt[words[i - 1]][words[i]] += 1
        biCnt['<s>'][words[0]] += 1
        biCnt[words[-1]]['</s>'] += 1
        
print(len(biCnt))
assert len(biCnt) == len(uniCnt) - 1
assert sum(biCnt['<s>'].values()) == uniCnt['<s>']
print(biCnt['<s>'])

53411
defaultdict(<class 'int'>, {'The': 10086, 'Could': 15, 'I': 406, 'One': 520, 'MADRID': 27, 'At': 623, 'Digitalization': 5, 'As': 1866, 'If': 1248, 'Admittedly': 17, 'That': 1072, 'And': 2191, 'Putin': 265, 'Second': 207, 'Rather': 351, 'By': 542, 'It': 1958, 'After': 667, 'Two': 69, 'Above': 38, 'Bolstered': 1, 'Improvement': 2, 'In': 4220, 'Project': 134, 'Even': 476, 'Human': 17, 'More': 295, 'She': 84, 'This': 2312, 'Our': 353, 'Confronted': 5, 'PRAGUE': 9, 'Asset': 1, 'Most': 238, 'Europe': 226, 'But': 5457, 'Likewise': 215, 'We': 788, 'Yet': 605, 'Islamist': 4, 'Now': 281, 'Some': 401, 'These': 571, 'Moreover': 753, 'History': 31, 'Farmers': 8, 'Finally': 234, 'Neither': 45, 'Many': 370, 'Few': 32, 'Still': 110, 'His': 169, 'Twenty-first-century': 1, 'Machiavelli': 3, 'Liberals': 4, 'Earlier': 24, 'No': 230, 'Last': 112, 'They': 800, 'Tree': 1, 'Following': 178, 'A': 1278, 'Multiple': 6, 'Over': 135, 'Consider': 133, 'With': 600, 'FREETOWN': 2, '“': 254, 'Given': 311, 'So': 

#### Discussion

### 1.2 $n$-gram Language Modeling

#### Code

In [41]:
# unigram
class Unigram():
    def train(self, data):
        pass

# calc prob for unigram
prob_d = {}
for word, cnt in voc.items():
    prob_d[word] = cnt / word_cnt
print(prob_d['<s>'])
print(prob_d['unk'])

# prob for all
# perp = 1
# for prob in prob_d.values():
#     perp *= prob ** (-1/len(prob_d))

# Calculate the perp of each sentence and then calculate the mean
perps = []
with open('data/lm/dev.txt', encoding='utf-8') as fd:
    lines = fd.readlines()
    for line in lines:
        words = line.split(' ')
        perp = 1
        for word in words:
            if word in prob_d:
                perp *= prob_d[word] ** (-1/len(words))
            else:
                perp *= prob_d['unk'] ** (-1/len(words))
        # perp = perp ** (-1/len(words))
        perps.append(perp)
        # print(perp)
minPerp = sum(perps) / len(perps)
minPerp

0.03750499871310973
0.01770329701755562


1214.6577034119832

这里的值和PPT中的一致

In [43]:
# bigram

from collections import defaultdict
# calc conditional perp
biProbs = defaultdict(lambda: defaultdict(int))
for wi in biCnt:
    wiCnt = 0
    for _, cnt in biCnt[wi].items():
        wiCnt += cnt
    for wj, cnt in biCnt[wi].items():
        biProbs[wi][wj] = cnt / wiCnt
print(biProbs['<s>'])
print(biProbs['The'])
assert sum(biProbs['<s>'].values()) - 1 < 1e-5
assert sum(biProbs['The'].values()) - 1 < 1e-5

# Calculate the perp of each sentence and then calculate the mean
def calc_perp(biProbs):
    perps = []
    with open(f'data/lm/{dataset}.txt', encoding='utf-8') as fd:
        lines = fd.readlines()
        for line in lines:
            words = line.split(' ')
            perp = biProbs['<s>'][words[0]] ** (-1/len(words))
            for i in range(1, len(words)):
                if biProbs[words[i - 1]][words[i]] != 0:
                    perp *= biProbs[words[i - 1]][words[i]] ** (-1/len(words))
                
            perp *= biProbs[words[-1]]['</s>'] ** (-1/len(words))
            # perp = perp ** (-1/len(words))
            # print(perp)
            perps.append(perp)
    return perps
    
perps = calc_perp(biProbs)
print(perps[:10])

defaultdict(<class 'int'>, {'The': 0.126075, 'Could': 0.0001875, 'I': 0.005075, 'One': 0.0065, 'MADRID': 0.0003375, 'At': 0.0077875, 'Digitalization': 6.25e-05, 'As': 0.023325, 'If': 0.0156, 'Admittedly': 0.0002125, 'That': 0.0134, 'And': 0.0273875, 'Putin': 0.0033125, 'Second': 0.0025875, 'Rather': 0.0043875, 'By': 0.006775, 'It': 0.024475, 'After': 0.0083375, 'Two': 0.0008625, 'Above': 0.000475, 'Bolstered': 1.25e-05, 'Improvement': 2.5e-05, 'In': 0.05275, 'Project': 0.001675, 'Even': 0.00595, 'Human': 0.0002125, 'More': 0.0036875, 'She': 0.00105, 'This': 0.0289, 'Our': 0.0044125, 'Confronted': 6.25e-05, 'PRAGUE': 0.0001125, 'Asset': 1.25e-05, 'Most': 0.002975, 'Europe': 0.002825, 'But': 0.0682125, 'Likewise': 0.0026875, 'We': 0.00985, 'Yet': 0.0075625, 'Islamist': 5e-05, 'Now': 0.0035125, 'Some': 0.0050125, 'These': 0.0071375, 'Moreover': 0.0094125, 'History': 0.0003875, 'Farmers': 0.0001, 'Finally': 0.002925, 'Neither': 0.0005625, 'Many': 0.004625, 'Few': 0.0004, 'Still': 0.001375,

TODO:上面代码补充注释，添加assert保证计算正确，绘图方便后续理解和laplace
这里的结果偏小，怀疑是因为数据集太小的原因，train的结果比dev的结果更大，因为train数据集更大

#### Discussion

### 1.3 Smoothing

#### 1.3.1 Add-one (Laplace) smoothing

##### Code

In [ ]:
from copy import deepcopy
def getOovLaplace():
    oovLaplace = deepcopy(biOov)
    # 计算新的条件概率
    biLaplaceProbs = defaultdict(lambda: defaultdict(int))
    for wi in oovLaplace:
        # add one
        for wj in oovLaplace:
            oovLaplace[wi][wj] += 1
        wiCnt = 0
        for wj, cnt in oovLaplace[wi].items():
            wiCnt += cnt
        for wj, cnt in oovLaplace[wi].items():
            biLaplaceProbs[wi][wj] = cnt / wiCnt
        print(oovLaplace[wi])
        return
biLaplaceProbs = getOovLaplace()

defaultdict(<class 'int'>, {'The': 2, 'multifaceted': 2, 'diverse': 2, 'articulated': 2, 'But': 2, 'relevant': 2, 'and': 2, 'the': 4, 'different': 2, 'regional': 2, 'of': 3, 's': 2, 'differing': 2, ',': 2, 'combine': 2, 'pessimistic': 2, 'scientifically-founded': 2, 'main': 2, 'strategic': 2, 'offer': 2, 'informed': 2, 'alternative': 2, 'even': 2, 'in': 2, 'perspectives': 1, 'proponents': 1, 'opponents': 1, 'are': 1, 'so': 1, 'polarized': 1, 'that': 1, 'it': 1, 'is': 1, 'hard': 1, 'not': 1, 'to': 1, 'be': 1, 'utterly': 1, 'confused': 1, 'about': 1, 'likely': 1, 'consequences': 1, '.\n': 1, '<s>': 1, 'history': 1, '–': 1, 'particular': 1, 'behavior': 1, 'Joseph': 1, 'Stalin': 1, '’': 1, 'Soviet': 1, 'Union': 1, 'after': 1, 'World': 1, 'War': 1, 'II': 1, 'give': 1, 'Western': 1, 'leaders': 1, 'insights': 1, 'they': 1, 'need': 1, 'mitigate': 1, 'threat': 1, '?\n': 1, 'suggest': 1, 'a': 1, 'temporary': 1, 'tax': 1, 'cut': 1, 'for': 1, 'two': 1, 'reasons': 1, 'possible': 1, 'explanation': 1

In [ ]:
# 计算新的perplexity
perps = calc_perp(biLaplaceProbs)
print(perps[:10])

##### Discussion

对所有的ovv进行laplace耗时非常久；没有对空白值加1时，相较于之前的模型，perplexity有适当减小
可以对每一句的word进行laplace变换

##### Optional: Add-k smoothing

###### Code

###### Discussion

#### 1.3.2 Linear Interpolation

##### Code

In [ ]:
# 如何实现trigram

# 计算条件概率

In [ ]:
# 实验不同的超参数


##### Discussion

##### Optional: Optimization

###### Discussion

## 2 Preposition Prediction

In [ ]:
!wget -O dev.in https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/prep/dev.in
!wget -O dev.out https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/prep/dev.out

'wget' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
'wget' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
